How do we **measure the quality or value of the prediction** in the selected business or
science problem? What will be the quantitative score? **How does the quantitative score
reflect the quality or value of the prediction?** How does the (possibly asymmetric)
prediction error convert into cost or decreased KPI?